In [3]:
!pip install pyspark
!pip install sparknlp

     |████████████████████████████████| 317.0 MB 18 kB/s  eta 0:00:011    |██████▊                         | 66.5 MB 4.6 MB/s eta 0:00:55     |██████▊                         | 66.9 MB 4.6 MB/s eta 0:00:55     |█████████████▍                  | 132.8 MB 4.0 MB/s eta 0:00:46     |█████████████▉                  | 136.9 MB 2.7 MB/s eta 0:01:06     |██████████████▊                 | 145.8 MB 5.0 MB/s eta 0:00:35     |████████████████▊               | 165.5 MB 5.8 MB/s eta 0:00:26     |████████████████▉               | 167.2 MB 7.5 MB/s eta 0:00:20     |█████████████████████████▏      | 248.8 MB 4.7 MB/s eta 0:00:15 |██████████████████████████▉     | 266.1 MB 4.7 MB/s eta 0:00:11     |███████████████████████████▍    | 270.9 MB 1.0 MB/s eta 0:00:46     |███████████████████████████▌    | 272.6 MB 1.0 MB/s eta 0:00:44     |████████████████████████████    | 276.6 MB 6.5 MB/s eta 0:00:07
     |████████████████████████████████| 200 kB 3.1 MB/s eta 0:00:01
Using legacy 'setup.py install' for pysp

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType, DataType,ArrayType
from pyspark.sql.functions import udf, struct
from pyspark.ml import Pipeline
from IPython.core.display import display, HTML
import re


/var/folders/fv/z4h0lznx44n3wybc34chcwhr0000gn/T/ipykernel_95845/3309149586.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

:: loading settings :: url = jar:file:/Users/isabeladias/.pyenv/versions/3.9.6/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/isabeladias/.ivy2/cache
The jars for the packages stored in: /Users/isabeladias/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f073322f-79e0-456f-9354-a94835d40390;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.4.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.5

Spark NLP version 5.4.0
Apache Spark version: 3.5.1
